# Data Pre-Processing for GPT condition

In [ ]:
import pandas as pd
import math

### Setup

In [ ]:
#Survey
qualtrics_df = pd.read_csv("/work/Results/Raw/GPT_Survey.csv", header=[1])
#Conversational Log
sheet_df = pd.read_csv("/work/Results/Raw/GPT_Chatlog.csv")

# drop qualtrics first row
qualtrics_df = qualtrics_df.drop(qualtrics_df.index[[0]])

# drop duplicate session IDs
qualtrics_df.drop_duplicates(subset=['session_id'])

#add conversation metrics
qualtrics_df["convo_conversation"] = ""
qualtrics_df["convo_group_id"] = ""
qualtrics_df["convo_AI_turns"] = ""
qualtrics_df["convo_human_turns"] = ""
qualtrics_df["convo_total_turns"] = ""

In [ ]:
qualtrics_df

,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,"Are you of Hispanic, Latino/a/x, or of Spanish origin? (one or more categories may be selected) - Selected Choice","Are you of Hispanic, Latino/a/x, or of Spanish origin? (one or more categories may be selected) - Prefer to self-describe - Text",assigned_category,assigned_group,session_id,convo_conversation,convo_group_id,convo_AI_turns,convo_human_turns,convo_total_turns
1,2022-06-17 18:57:56,2022-06-17 19:10:41,IP Address,*******,100,765,True,2022-06-17 19:10:41,R_sT4iHsDd1L3MO4N,*******,...,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,NaN,3,2022-06-18T00:58:33.311Z,,,,,
2,2022-06-17 19:06:58,2022-06-17 19:30:44,IP Address,*******,100,1426,True,2022-06-17 19:30:44,R_3HzsyHEWtJTUPi1,*******,...,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,NaN,2,2022-06-18T01:08:23.936Z,,,,,
3,2022-06-17 19:37:24,2022-06-17 19:55:51,IP Address,*******,100,1106,True,2022-06-17 19:55:51,R_qJhnyVIDIs4CJYR,*******,...,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,NaN,1,2022-06-18T01:40:50.676Z,,,,,
4,2022-06-17 19:55:10,2022-06-17 20:10:32,IP Address,*******,100,921,True,2022-06-17 20:10:32,R_2fBsUpW2LcXQ7SP,*******,...,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,NaN,3,2022-06-18T01:56:45.963Z,,,,,
5,2022-06-17 19:55:10,2022-06-17 20:10:43,IP Address,*******,100,933,True,2022-06-17 20:10:43,R_2V9McykguniYrKX,*******,...,"No, not of Hispanic, Latino/a/x, or Spanish or...",NaN,NaN,3,2022-06-18T01:57:09.013Z,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2022-06-24 10:02:35,2022-06-24 10:07:16,IP Address,*******,37,280,False,2022-07-01 10:07:19,R_30ki51zTnNU6VqI,*******,...,NaN,NaN,NaN,2,2022-06-24T16:04:10.776Z,,,,,
194,2022-06-24 10:08:15,2022-06-24 10:22:20,IP Address,*******,54,844,False,2022-07-01 10:22:22,R_RfOYQv2N5zOX8bf,*******,...,NaN,NaN,NaN,2,2022-06-24T16:10:41.804Z,,,,,
195,2022-06-24 10:21:51,2022-06-24 10:24:02,IP Address,*******,11,131,False,2022-07-01 10:24:02,R_3sp4TEOPY8SQRLc,*******,...,NaN,NaN,NaN,2,NaN,,,,,
196,2022-06-24 10:27:56,2022-06-24 10:32:19,IP Address,*******,17,262,False,2022-07-01 10:32:19,R_3KYONKOYQtjLS4K,*******,...,NaN,NaN,NaN,3,NaN,,,,,


### Data Integration

In [ ]:
#Merge data

ext_id = []

# get session IDs from Qualtrics data
for j in qualtrics_df.index:
    
    #get qualtrics session_id
    qualtrics_session_id = qualtrics_df.loc[j, "session_id"]

    if qualtrics_session_id in ext_id:
        continue
    else: 
        ext_id.append(qualtrics_session_id)
    
    #Add all the conversation together
    converstion = ""
    group_id = ""
    AI_turns = 0
    human_turns = 0

    #iterate to sheet_id
    for i in sheet_df.index:
        if (sheet_df.loc[i, "session_id_post"]) == qualtrics_session_id:
            
            Human_text = str(sheet_df.loc[i, "user_content_post"])
            if (Human_text != "nan"):
                converstion =  converstion + " >Human: " + Human_text
                human_turns = human_turns+1
            AI_text = str(sheet_df.loc[i, "AI_content_post"])
            if (AI_text != "nan"):
                converstion =  converstion + " >AI: " + AI_text
                AI_turns = AI_turns+1
        
            tmp_group_id = str(sheet_df.loc[i, "group_id"])
            if (tmp_group_id != "nan"):
                group_id = tmp_group_id

    #update the convo metrics
    qualtrics_df.loc[j, "convo_conversation"] = converstion
    qualtrics_df.loc[j, "convo_group_id"] = group_id
    qualtrics_df.loc[j, "convo_AI_turns"] = AI_turns
    qualtrics_df.loc[j, "convo_human_turns"] = human_turns
    qualtrics_df.loc[j, "convo_total_turns"] = human_turns + AI_turns

In [ ]:
#Update the label in assigned_group
for j in qualtrics_df.index:
    qualtrics_assigned_group = qualtrics_df.loc[j, "assigned_group"]
    if (qualtrics_assigned_group == "1"):
        qualtrics_df.loc[j, "assigned_group"] = "No Motive"
    if (qualtrics_assigned_group == "2"):
        qualtrics_df.loc[j, "assigned_group"] = "Caring Motive"
    if (qualtrics_assigned_group == "3"):
        qualtrics_df.loc[j, "assigned_group"] = "Manipulative Motive"

In [ ]:
#Update the label in convo_group_id
for j in qualtrics_df.index:
    qualtrics_assigned_group = qualtrics_df.loc[j, "convo_group_id"]
    if (qualtrics_assigned_group == "AI with no motives"):
        qualtrics_df.loc[j, "convo_group_id"] = "No Motive"
    if (qualtrics_assigned_group == "AI with caring motives"):
        qualtrics_df.loc[j, "convo_group_id"] = "Caring Motive"
    if (qualtrics_assigned_group == "AI with manipulative motives"):
        qualtrics_df.loc[j, "convo_group_id"] = "Manipulative Motive"

In [ ]:
for j in qualtrics_df.index:
    df_perception = qualtrics_df.loc[j, "From your own experience, what do you think the motive of the agent was? - Selected Choice"]
    if (df_perception == "Caring motives"):
        qualtrics_df.loc[j, "Perceived Motives"] = "Caring Motive"
    elif (df_perception == "No motive"):
        qualtrics_df.loc[j, "Perceived Motives"] = "No Motive"
    elif (df_perception == "Manipulative/malicious motives"):
        qualtrics_df.loc[j, "Perceived Motives"] = "Manipulative Motive"


In [ ]:
qualtrics_df['UTAUT Performance'] = qualtrics_df[['Please rank the following statements. - This agent would be useful in daily life.',
                                'Please rank the following statements. - Using the agent would increase my chances of achieving things that are important to me.', 
                                'Please rank the following statements. - Using the agent would help me accomplish things more quickly.', 
                                'Please rank the following statements. - Using the agent would increase my productivity.']].mean(axis=1)
qualtrics_df['UTAUT Effort'] = qualtrics_df[['Please rank the following statements. - Learning how to talk to the agent was easy for me.', 
                            'Please rank the following statements. - My interaction with the agent was clear and understandable.', 
                            'Please rank the following statements. - The agent was easy to make use of.', 
                            'Please rank the following statements. - It was easy for you to become skillful at making use of the agent.']].mean(axis=1)
qualtrics_df['UTAUT Hedonic'] = qualtrics_df[['Please rank the following statements. - Conversing with the agent is fun.', 
                            'Please rank the following statements. - Conversing with the agent is enjoyable.', 
                            'Please rank the following statements. - Conversing with the agent is entertaining.']].mean(axis=1)

In [ ]:
#Update column name
qualtrics_df.columns = qualtrics_df.columns.str.replace(".", "")
qualtrics_df.columns = qualtrics_df.columns.str.replace("-", "")
qualtrics_df.columns = qualtrics_df.columns.str.replace("(", "")
qualtrics_df.columns = qualtrics_df.columns.str.replace(")", "")
qualtrics_df.columns = qualtrics_df.columns.str.replace(" 1 being very low and 20 being very high  ", "")
qualtrics_df.columns = qualtrics_df.columns.str.replace("Please rank the following statements  ", "Agent: ")
qualtrics_df.columns = qualtrics_df.columns.str.replace("Please rank the following statements according to how much you agree or disagree with them These pertain to your sentiment in regards to AI artificial intelligence  ", "AI: ")
qualtrics_df.columns = qualtrics_df.columns.str.replace("Please rank the following statements according to how much you agree or disagree with them These relate to your experience with the agent  ", "Agent: ")
qualtrics_df.columns = qualtrics_df.columns.str.replace("Task Load Index  please rate the following items,", "")


/tmp/ipykernel_90/3388320141.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  qualtrics_df.columns = qualtrics_df.columns.str.replace(".", "")
/tmp/ipykernel_90/3388320141.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  qualtrics_df.columns = qualtrics_df.columns.str.replace("(", "")
/tmp/ipykernel_90/3388320141.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  qualtrics_df.columns = qualtrics_df.columns.str.replace(")", "")


### Cleaned Composite Data

In [ ]:
qualtrics_df

,Start Date,End Date,Response Type,IP Address,Progress,Duration in seconds,Finished,Recorded Date,Response ID,Recipient Last Name,...,session_id,convo_conversation,convo_group_id,convo_AI_turns,convo_human_turns,convo_total_turns,Perceived Motives,UTAUT Performance,UTAUT Effort,UTAUT Hedonic
1,2022-06-17 18:57:56,2022-06-17 19:10:41,IP Address,*******,100,765,True,2022-06-17 19:10:41,R_sT4iHsDd1L3MO4N,*******,...,2022-06-18T00:58:33.311Z,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,2,1,3,No Motive,1388.75,1388.75,185.000000
2,2022-06-17 19:06:58,2022-06-17 19:30:44,IP Address,*******,100,1426,True,2022-06-17 19:30:44,R_3HzsyHEWtJTUPi1,*******,...,2022-06-18T01:08:23.936Z,">AI: I am Melu, a mental health companion. I ...",Caring Motive,37,36,73,Caring Motive,1361.00,1944.00,259.000000
3,2022-06-17 19:37:24,2022-06-17 19:55:51,IP Address,*******,100,1106,True,2022-06-17 19:55:51,R_qJhnyVIDIs4CJYR,*******,...,2022-06-18T01:40:50.676Z,">AI: I am Melu, a mental health companion. I ...",No Motive,27,26,53,Caring Motive,1666.50,1666.50,222.000000
4,2022-06-17 19:55:10,2022-06-17 20:10:32,IP Address,*******,100,921,True,2022-06-17 20:10:32,R_2fBsUpW2LcXQ7SP,*******,...,2022-06-18T01:56:45.963Z,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,20,19,39,Caring Motive,805.25,1666.50,110.666667
5,2022-06-17 19:55:10,2022-06-17 20:10:43,IP Address,*******,100,933,True,2022-06-17 20:10:43,R_2V9McykguniYrKX,*******,...,2022-06-18T01:57:09.013Z,">AI: I am Melu, a mental health companion. I ...",Manipulative Motive,11,10,21,Caring Motive,1058.75,838.25,40.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2022-06-24 10:02:35,2022-06-24 10:07:16,IP Address,*******,37,280,False,2022-07-01 10:07:19,R_30ki51zTnNU6VqI,*******,...,2022-06-24T16:04:10.776Z,">AI: I am Melu, a mental health companion. I ...",Caring Motive,4,3,7,NaN,NaN,NaN,NaN
194,2022-06-24 10:08:15,2022-06-24 10:22:20,IP Address,*******,54,844,False,2022-07-01 10:22:22,R_RfOYQv2N5zOX8bf,*******,...,2022-06-24T16:10:41.804Z,">AI: I am Melu, a mental health companion. I ...",Caring Motive,20,19,39,NaN,NaN,NaN,NaN
195,2022-06-24 10:21:51,2022-06-24 10:24:02,IP Address,*******,11,131,False,2022-07-01 10:24:02,R_3sp4TEOPY8SQRLc,*******,...,NaN,,,,,,NaN,NaN,NaN,NaN
196,2022-06-24 10:27:56,2022-06-24 10:32:19,IP Address,*******,17,262,False,2022-07-01 10:32:19,R_3KYONKOYQtjLS4K,*******,...,NaN,,,,,,NaN,NaN,NaN,NaN


### Filtering

In [ ]:
#filter technical issue
final_df = qualtrics_df[qualtrics_df['Did you have any technical difficulties?  Selected Choice'] == 'No'] 
print("Original: " + str(qualtrics_df.shape))
print("Remaining: " + str(final_df.shape))

Original: (197, 94)
Remaining: (174, 94)


In [ ]:
#filter conversation < 4
final_df = final_df[final_df['convo_total_turns'] > 4] 
print("Original: " + str(qualtrics_df.shape))
print("Remaining: " + str(final_df.shape))

Original: (197, 94)
Remaining: (169, 94)


In [ ]:
#Attention check#1
final_df = final_df[final_df['What kind of agent are you conversing with?'] == 'A mental health companion'] 
print("Original: " + str(qualtrics_df.shape))
print("Remaining: " + str(final_df.shape))

Original: (197, 94)
Remaining: (169, 94)


In [ ]:
#Attention check#2
final_df = final_df[final_df['What is the scenario you are in?'] == 'You are conversing with a mental health conversational agent to determine whether you want to recommend this for your friend.'] 
print("Original: " + str(qualtrics_df.shape))
print("Remaining: " + str(final_df.shape))


Original: (197, 94)
Remaining: (169, 94)


In [ ]:
##Attention check#3 mismatch goal/motive
final_df["self_reported_motives"] = final_df["What is the motive/goal of the AI you are assigned to chat with?"].fillna('').astype(str) 

for j in final_df.index:
    self_report_column = final_df.loc[j, "self_reported_motives"]
    if (self_report_column == "No motives"):
        final_df.loc[j, "self_reported_motives"] = "No Motive"
    if (self_report_column == "Caring motives, to improve mental health"):
        final_df.loc[j, "self_reported_motives"] = "Caring Motive"
    if (self_report_column == "Manipulative motives, to get you to purchase its service"):
        final_df.loc[j, "self_reported_motives"] = "Manipulative Motive"

#Remove the mismatch goal/motive
final_df = final_df[final_df['self_reported_motives'] == final_df['assigned_group']]
print(qualtrics_df.shape)
print(final_df.shape)

(197, 94)
(161, 95)


In [ ]:
#Remove the mismatch ID
final_df = final_df[final_df['convo_group_id'] == final_df['assigned_group']]
print(qualtrics_df.shape)
print(final_df.shape)


(197, 94)
(160, 95)


In [ ]:
#Check unique Prolific ID
len(final_df["What is your Prolific ID?"].unique())

160

In [ ]:
#Check unique Session ID
len(final_df["session_id"].unique())

160

In [ ]:
final_df.to_csv("/work/Results/Processed/GPT_Processed_Data.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1f3ea62d-9e62-461e-8c65-1caf654509ee' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>